Scraping : extract data from the websites
Crawling : find the further links

THIS IS SPECIFICALLY USED FOR THE WEBSITE SCRAPING

installing scrapy :

pip install scrapy

        

THIS IS USED FOR PDF CRAWLING

For extracting the data from a pdf file
1) we need text
2) we need tables
3) we need images

the libraries required:

pip install pdfminer.six



This here is an example of how the texts and the different elements can be extracted from a pdf file

We can use re.compile in order to modify the output.


In [ ]:
import re
import io
from pdfminer.high_level import extract_pages, extract_text


for page_layout in extract_pages('chemdataextractor.pdf'):
    for element in page_layout:
        print(element)



In [ ]:
text = extract_text('PaperData\\file1.pdf')
print(text)
from chemdataextractor import Document
from chemdataextractor.nlp.tokenize import ChemWordTokenizer
from collections import Counter
import re

# Load your chemical text data


# Parse the document
doc = Document(text)

# Tokenize the text
tokenizer = ChemWordTokenizer()
tokens = tokenizer.tokenize(doc.text)

# Filter tokens
filtered_tokens = [token for token in tokens if re.match(r'^[a-zA-Z\-]+$', token)]

# Create a keyword list
keyword_counts = Counter(filtered_tokens)
keywords = keyword_counts.most_common()

# Display the keyword list
for keyword, count in keywords:
    print(f"{keyword}: {count}")





# print(text)

We have to do 2 things, firs of all take the keywords which have come up in our data list upto now

Extracting the images from a pdf

pip install PyMuPDF
pip install pillow



The program below will find out all of the 

In [ ]:
import fitz #pymupdf
import PIL.Image #pllow
import io

pdf = fitz.open('pdfsample.pdf')
counter =1 
for i in range(len(pdf)):
    page = pdf[i]
    images = page.get_images()
    for image in images:
        base_img = pdf.extract_image(image[0])
        image_data = base_img["image"]
        img = PIL.Image.open(io.BytesIO(image_data))
        extension = base_img["ext"]
        img.save(open(f"image{counter}.{extension}", "wb"))
        counter += 1


this is the code for reading all of the pdf files in a directory and then finding the given keywords and it gives the list of pages and context where each keyword is found


In [2]:
import os
import fitz  # PyMuPDF
import re

def search_keywords_in_pdf(pdf_path, keywords):
    # Open the PDF file
    document = fitz.open(pdf_path)
    results = {}

    # Iterate through each page
    for page_num in range(document.page_count):
        page = document.load_page(page_num)  # Load page
        text = page.get_text()  # Extract text from the page

        # Search for keywords in the text
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE):
                if keyword not in results:
                    results[keyword] = []
                # Collect the context and page number
                results[keyword].append((page_num + 1, text))

    return results

def search_keywords_in_directory(directory_path, keywords):
    # List all files in the directory
    for file_name in os.listdir(directory_path):
        # Check if the file is a PDF
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, file_name)
            print(f"Searching in {file_name}...\n")
            # opening a file with same name as the pdf
            # file = open(f"{file_name}.txt", 'w')

            # Search for keywords in the current PDF

            # Print the results
            for keywordList in keywords:
                results = search_keywords_in_pdf(pdf_path, keywordList)
                for keyword, occurrences in results.items():
                    print(f"Keyword '{keyword}' found in {file_name}:\n\n")
                    for page_num, context in occurrences:
                        print(f"..{keyword} on Page {page_num}:\n")
                        # Print a snippet of the context around the keyword
                        start_idx = context.lower().find(keyword.lower())
                        snippet = context[max(0, start_idx-200):start_idx+len(keyword)+200]
                        print(f"    ..{snippet}..\n\n\n")
        print('-----------------------------------------------------------------------------------------------------------------------------------------------')

# Example usage

directory_path = 'C:\\Users\\addys\\OneDrive\\Documents\\PythonProjectIITM\\PaperData'
# These ar the sections after which the keyword search will begin
req_sections=['']

## these are the categorical keywords which shall be searched in the files in sections 
electrode = ['electrode', 'Activated','carbon','Activated Carbon', 'Stainless Steel']  # List of keywords to search for
cyclicVoltametry= ['potential', 'window', 'potential window', 'specific capacitance', 'current density', 'scan rates']
units=['area','cmxcm', 'cm2', 'voltage', 'V', 'normal','N','molar', 'M', 'A/cm2', 'F/g','mV/s' ]

keywords = [electrode, cyclicVoltametry, units]
search_keywords_in_directory(directory_path, keywords)


Searching in chemdataextractor.pdf...

Keyword 'potential' found in chemdataextractor.pdf:


..potential on Page 2:

    ..ry literature presents an attractive and tractable target for this automated
extraction as it is typically comprised of formulaic, data-rich language that is well-suited for
machine analysis with the potential for high recall and precision. The extracted chemical
information can be used to create and populate databases of chemical structures, properties
and observations, opening up new avenues for discover..



Keyword 'window' found in chemdataextractor.pdf:


..window on Page 11:

    ..proves performance across a wider range of subject areas and document
domains (such as captions) that are not well-covered by the training corpora.
Table 1: Features used in the POS tagger. A context window is used, such that some features
for the token at index i are derived from the token text (w) of surrounding tokens.
Feature
Context
Description
Word
wi−2, wi−1, wi, wi+1, wi+2

# applying the new method

One method of solving the issue is to put up a filter and only start the search for the related keywords after having found only the required sections in the keywords list!!
We have to just add another layer of filters
 

In [2]:
import os
import fitz  # PyMuPDF
import re

def search_keywords_in_pdf(pdf_path, keywords):
    # Open the PDF file
    document = fitz.open(pdf_path)
    results = {}

    # Iterate through each page
    for page_num in range(document.page_count):
        page = document.load_page(page_num)  # Load page
        text = page.get_text()  # Extract text from the page

        # Search for keywords in the text
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE):
                if keyword not in results:
                    results[keyword] = []
                # Collect the context and page number
                results[keyword].append((page_num + 1, text))

    return results

def search_keywords_in_directory(directory_path, keywords):
    # List all files in the directory
    for file_name in os.listdir(directory_path):
        # Check if the file is a PDF
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, file_name)
            print(f"Searching in {file_name}...\n")
            # opening a file with same name as the pdf
            # file = open(f"{file_name}.txt", 'w')

            # Search for keywords in the current PDF
            results = search_keywords_in_pdf(pdf_path, keywords)

            # Print the results
            for keyword, occurrences in results.items():
                print(f"Keyword '{keyword}' found in {file_name}:\n\n")
                for page_num, context in occurrences:
                    print(f"  Page {page_num}:\n")
                    # Print a snippet of the context around the keyword
                    start_idx = context.lower().find(keyword.lower())
                    snippet = context[max(0, start_idx-200):start_idx+len(keyword)+200]
                    print(f"    ..{snippet}..\n\n\n")
        print('-----------------------------------------------------------------------------------------------------------------------------------------------')

# Example usage

directory_path = 'C:\\Users\\addys\\OneDrive\\Documents\\PythonProjectIITM\\PaperData'
# These ar the sections after which the keyword search will begin
req_sections=['']

## these are the categorical keywords which shall be searched in the files in sections 
electrode = ['electrode', 'Activated','carbon','Activated Carbon', 'Stainless Steel']  # List of keywords to search for
cyclicVoltametry= ['potential', 'window', 'potential window', 'specific capacitance', 'current density', 'scan rates']
units=['area','cmxcm', 'cm2', 'voltage', 'V', 'normal','N','molar', 'M', 'A/cm2', 'F/g','mV/s' ]

keywords = [electrode, cyclicVoltametry, units]
for keywordList in keywords:
    search_keywords_in_directory(directory_path, keywordList)





AttributeError: module 'pypdf' has no attribute 'PdfReaderReader'

# Final but without the multiple lists of keyword has been created
adding the feature for using multiple keyword list will require for us to use only another for loop
not very difficult


In [3]:
import os
import fitz  # PyMuPDF
import re

## this is the function that does the search for the keywords for me 

def search_keywords_in_pdf(pdf_path, keywords):
    # Open the PDF file
    document = fitz.open(pdf_path)
    results = {}

    # Iterate through each page
    for page_num in range(document.page_count):
        page = document.load_page(page_num)  # Load page
        text = page.get_text()  # Extract text from the page

        # Search for keywords in the text
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE):
                if keyword not in results:
                    results[keyword] = []
                # Collect the context and page number
                results[keyword].append((page_num + 1, text))

    return results

## this is the function that enables me to change the way the search query is done in the function
def search_keywords_in_directory(directory_path, keywords, section_names):
    # List all files in the 
    ## this is opening up a file 
    for file_name in os.listdir(directory_path):
        # Check if the file is a PDF
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, file_name)
            mod_file_name = file_name[:-4]
            ## creating a new file to store the output for this file 
            new_file_loc = directory_path + f"/{mod_file_name}"+".txt"
            file = open(new_file_loc, 'w', encoding='utf-8')
            print(f"Searching in {file_name}...\n")
            file.writelines(f"results for {file_name}...\n")
            ## this is from where we are going to give the results for the various sections

            sections = search_keywords_in_pdf(pdf_path, section_names)
            index ={}
            for section, occurences in sections.items():
                for page_num, context in occurences:
                    if(context.lower().find(section.lower())!= -1):
                        section_idx = context.lower().find(section.lower())
                        file.writelines(f"Section {section} found.. on page {page_num} has index {section_idx}\n")
                        index[section] =section_idx
                        break

            print(index)
            # Search for keywords in the current PDF
            results = search_keywords_in_pdf(pdf_path, keywords)
            

            if(len(index)!=0):
                for section in index:
                    for keyword, occurrences in results.items():
                        # print(f"\n\n****Keyword '{keyword}' found in {file_name}: after beginning of section: {section}")
                        for page_num, context in occurrences:
                            start_idx = context.lower().find(keyword.lower())
                            if(start_idx>index[section]):
                                file.writelines(f"\n\n****Keyword '{keyword}' found in {file_name}: after beginning of section: {section}\n")
                                # print(f"\n Page {page_num}:")
                                file.write(f"\n\t Page {page_num}:")
                                # Print a snippet of the context around the keyword
                                snippet = context[max(0, start_idx-50):start_idx+len(keyword)+200]
                                # print(f"    ...{snippet}...")
                                file.writelines(f"    \t\t...{snippet}...\n\n")
                        
                

            # if there are no sections found, then we will assume that the paper is a relevant one and we will be taking all of the content
            else:
            # Print the results
                for keyword, occurrences in results.items():
                    file.writelines("\n\n\tATTENTION!! The paper couldnt be detected into sections the whole paper is searched for keywords.\n\n\t")
                    file.writelines(f"\n\n****Keyword '{keyword}' found in {file_name}:\n\n")
                    for page_num, context in occurrences:
                        file.writelines(f"\n \tPage {page_num}:")
                        # Print a snippet of the context around the keyword
                        start_idx = context.lower().find(keyword.lower())
                        snippet = context[max(0, start_idx-50):start_idx+len(keyword)+200]
                        file.writelines(f"   \t\t ...{snippet}...\n\n")
                    # print()
            file.close()

# Example usage


def main():
    directory_path = 'C:\\Users\\addys\\OneDrive\\Documents\\PythonProjectIITM\\PaperData'

    electrode = ['electrode', 'Activated','carbon','Activated Carbon', 'Stainless Steel']  # List of keywords to search for
    cyclicVoltametry= ['potential', 'window', 'potential window', 'specific capacitance', 'current density', 'scan rates']
    units=['area','cmxcm', 'cm2', 'voltage', 'V', 'normal','N','molar', 'M', 'A/cm2', 'F/g','mV/s' ]

    section_names = ["Experimental","Results and discussion","Conclusion"]
    keywordList = electrode + cyclicVoltametry + units 
    search_keywords_in_directory(directory_path, keywordList, section_names)

main()





Searching in chemdataextractor.pdf...

{'Experimental': 453}
Searching in file1.pdf...

{'Experimental': 5494, 'Results and discussion': 3058}
Searching in file10.pdf...

{'Conclusion': 5339, 'Experimental': 4986}
Searching in file11.pdf...

{'Experimental': 2828, 'Results and discussion': 666}
Searching in file12.pdf...

{'Experimental': 3717, 'Results and discussion': 1530}
Searching in file13.pdf...

{'Results and discussion': 1738, 'Conclusion': 717}
Searching in file14.pdf...

{'Experimental': 6193, 'Conclusion': 3413}
Searching in file16.pdf...

{'Experimental': 2909, 'Results and discussion': 5904, 'Conclusion': 2357}
Searching in file17.pdf...

{'Experimental': 1999, 'Results and discussion': 952}
Searching in file18.pdf...

{'Experimental': 5494, 'Results and discussion': 3058}
Searching in file19.pdf...

{'Experimental': 773, 'Results and discussion': 4424, 'Conclusion': 5490}
Searching in file2.pdf...

{'Experimental': 1899, 'Results and discussion': 1242, 'Conclusion': 1443